Способы фильтрации данных в Pandas

https://highload.today/python-pandas/

Содержание:
1. Общая информация и подготовка данных
2. Фильтрация данных pandas
3. Вспомогательные функции для фильтрации
4. Фильтруем пропуски в данных
5. Построение аналитики по сгруппированным данным
6. Как фильтровать данные без использования пакета pandas
В заключение (бонус видео)

In [35]:
import pandas as pd

1. Общая информация и подготовка данных

In [36]:
df = pd.DataFrame({'name': ['Taras','Ivan','Stas'], 'surname': ['Shevchenko','Kozak','Gerb'], 'height': [184,170,176]})
# смотрим что внутри df
df

,name,surname,height
0,Taras,Shevchenko,184
1,Ivan,Kozak,170
2,Stas,Gerb,176


2. Фильтрация данных pandas

Метод 1: DataFrame Way

In [37]:
df_tall = df[df['height'] > 175]

In [38]:
df_tall[['name','surname']]

,name,surname
0,Taras,Shevchenko
2,Stas,Gerb


Множественные условия фильтрации

In [39]:
df_tall_named = df[((df['height'] > 175) | (df['name'] == 'Taras')) & (df['surname'] == 'Shevchenko')]
df_tall_named

,name,surname,height
0,Taras,Shevchenko,184


Метод 2: Query Function

In [40]:
max_height = 175
chosen_name = 'Taras'
df_tall_named_sec = df.query('height > @max_height & name == @chosen_name')
df_tall_named_sec

,name,surname,height
0,Taras,Shevchenko,184


In [41]:
df_tall_named_trd = df.query('height < 175 & `name` != "Taras Shevchenko"')
df_tall_named_trd

,name,surname,height
1,Ivan,Kozak,170


Метод 3: loc function

In [42]:
# получаем доступ к строке с индексом 0:
df.loc[0]

name            Taras
surname    Shevchenko
height            184
Name: 0, dtype: object

In [43]:
df.loc[0]["height"]

184

In [44]:
df.loc[0][2]

184

In [45]:
df.loc[[0,0,1,1,2,2,2]]

,name,surname,height
0,Taras,Shevchenko,184
0,Taras,Shevchenko,184
1,Ivan,Kozak,170
1,Ivan,Kozak,170
2,Stas,Gerb,176
2,Stas,Gerb,176
2,Stas,Gerb,176


In [46]:
# выделяем строки с 1 по 2:
df.loc[1:2]

,name,surname,height
1,Ivan,Kozak,170
2,Stas,Gerb,176


In [47]:
# выделяем все строки в указанных в списке столбцах
df.loc[:,["name","surname"]]

,name,surname
0,Taras,Shevchenko
1,Ivan,Kozak
2,Stas,Gerb


In [48]:
loc_df_sec = df.loc[df.height > 175]
loc_df_sec

,name,surname,height
0,Taras,Shevchenko,184
2,Stas,Gerb,176


In [49]:
loc_df_thr = df.loc[df.height > 175,["name","surname"]]
loc_df_thr

,name,surname
0,Taras,Shevchenko
2,Stas,Gerb


In [50]:
df.loc[lambda df: df['height'] >= 170, "name"]

0    Taras
1     Ivan
2     Stas
Name: name, dtype: object

Метод 4: iloc function

In [51]:
df.iloc[0]

name            Taras
surname    Shevchenko
height            184
Name: 0, dtype: object

In [52]:
print(df.iloc[0].index)

Index(['name', 'surname', 'height'], dtype='object')


In [53]:
print(df.iloc[0].index[0])

name


In [54]:
df.iloc[0][0]

'Taras'

In [55]:
# первый аргумент — числовой индекс строки, второй — столбца:
df.iloc[0,0]

'Taras'

In [56]:
df.iloc[0:3,0:3]

,name,surname,height
0,Taras,Shevchenko,184
1,Ivan,Kozak,170
2,Stas,Gerb,176


In [57]:
df.iloc[lambda x: range(3), lambda x: range(3)]

,name,surname,height
0,Taras,Shevchenko,184
1,Ivan,Kozak,170
2,Stas,Gerb,176


In [58]:
df.iloc[[0,2,1]]

,name,surname,height
0,Taras,Shevchenko,184
2,Stas,Gerb,176
1,Ivan,Kozak,170


In [59]:
df.iloc[[0,0,1,1,2],[1,1,2]]

,surname,surname,height
0,Shevchenko,Shevchenko,184
0,Shevchenko,Shevchenko,184
1,Kozak,Kozak,170
1,Kozak,Kozak,170
2,Gerb,Gerb,176


In [60]:
df.iloc[:, df.columns.get_indexer(['name', 'height'])]

,name,height
0,Taras,184
1,Ivan,170
2,Stas,176


Разница между функциями loc и iloc
Основная разница двух методов состоит в том, что .loc[] работает с метками столбцов и строк, в то время как iloc[] работает с целочисленными позициями нужных элементов.

3. Вспомогательные функции для фильтрации

In [61]:
mask = df.iloc[:,2:3].stack().apply(lambda x: int(x) > 175).unstack()
# фильтруем всех, по указанной маске, подставляя в [] обернутый в () столбец с результатом apply, который у нас лежит в поле height
df[(mask.height)]

,name,surname,height
0,Taras,Shevchenko,184
2,Stas,Gerb,176


Использование побитового отрицания

In [62]:
df[~(mask.height)]

,name,surname,height
1,Ivan,Kozak,170


Проверка на вхождение символов в значение столбца

In [63]:
df[df['name'].str.contains('T')]

,name,surname,height
0,Taras,Shevchenko,184


In [64]:
df[df['name'].str.contains('v') | df['name'].str.startswith('S')]

,name,surname,height
1,Ivan,Kozak,170
2,Stas,Gerb,176


Использование .stack() и .apply() для формирования логической маски фильтра:

In [65]:
mask_surn = df.iloc[:,1:2].stack().apply(lambda x: len(x) >= 5).unstack()
df[(mask_surn.surname)]

,name,surname,height
0,Taras,Shevchenko,184
1,Ivan,Kozak,170


4. Фильтруем пропуски в данных

In [66]:
df=df.append({"name":"Tony","surname":"Stark","height":None},ignore_index=True)

In [67]:
df.isnull().sum()

name       0
surname    0
height     1
dtype: int64

In [68]:
df = df.dropna()

5. Построение аналитики по сгруппированным данным

In [69]:
h_calc_df = df[df.height > 175].groupby("height").height.aggregate(lambda x: (x-165)/165*100)
h_calc_df

height
176     6.666667
184    11.515152
Name: height, dtype: float64

6. Как фильтровать данные без использования пакета pandas

In [70]:
some_lst = [10, 9, 0, 8, 2, 6, 4, 3, 1, 7, 5]
small = [x for x in some_lst if x < 8]
small

[0, 2, 6, 4, 3, 1, 7, 5]

In [71]:
even = [x for x in some_lst if x%2==0]
even

[10, 0, 8, 2, 6, 4]

In [72]:
odd = [x for x in some_lst if x%2]
odd

[9, 3, 1, 7, 5]

In [73]:
small = filter(lambda x: x<8, some_lst)
list(small)

[0, 2, 6, 4, 3, 1, 7, 5]

In [74]:
sum(x for x in some_lst if x < 8)

28

In [75]:
def condition(x): 
    return x > 3 and x < 7 
filtered = [x for x in some_lst if condition(x)] 
filtered

[6, 4, 5]

In [76]:
some_lst = [10, 9, 0, 8, 2, 6, 4, 3, 1, 7, 5]
b_lst = [True, False, False, True, True, False, True, False, True, False,True]
res = [x for (x, boo) in zip(some_lst, b_lst) if boo]
res

[10, 8, 2, 4, 1, 5]